## Nombre
David Antonio Garzón Avendaño 
# Code Assigment 1

For this assignment you will use the following SVM implementation for classifying these datasets:

https://archive.ics.uci.edu/ml/datasets/banknote+authentication


https://archive.ics.uci.edu/ml/datasets/Occupancy+Detection+

You should:

1) Specify which Machine Learning problem are you solving.

2) Provide a short summary of the features and the labels you are working on.

3) Please answer the following questions: a) Are these datasets linearly separable? b) Are these datasets randomly chosen and c) The sample size is enough to guarantee generalization.

4) Provide an explanation how and why the code is working. You can add comments and/or formal explanations into the notebook.

5) Show some examples to illustrate that the method is working properly.

6) Provide quantitative evidence for generalization using the provided dataset.


#Respuestas


##Punto 1)



###Dataset Uno. 
Deseamos que dadas ciertas caracteristicas de una imagen de un billete, crear un modelo que sea capaz de identificar si un billete es falso o no. Para ello tenemos como datos:
* La varianza de la imagen transformada de wavelet
* La asimetría de la imagen transformada por wavelet
* La curtosis de la imagen transformada de wavelet
* La entropía de la imagen.

###Dataset Dos

Deseamos que dadas unas caracteristicas de un billete, crear un modelo que sea capaz de identificar identifica si hay personas o no dentro de un cuarto, para ello tenemos. Para ello tenemos como datos:
* Fecha y hora en que se tomó la medición
* Temperatura en grados Celsius
* Humedad relativa en porcentaje (%)
* Luz en lux
* CO2 en partes por millón (ppm)
* Razón de humedad en kg de vapor de agua por kg de aire

## Punto 2) 


###Dataset Uno. 

Los atributos describen las siguientes características de las imágenes transformadas por wavelet:

  * La varianza de la imagen transformada de wavelet. Es una medida de cuánto se extienden los valores de la imagen en el espacio de la transformada de wavelet.

  * La asimetría de la imagen transformada de wavelet. Es una medida de la falta de simetría en la distribución de los valores de la imagen en el espacio de la transformada de wavelet.

  * La curtosis de la imagen transformada de wavelet. Es una medida de la forma de la distribución de los valores de la imagen en el espacio de la transformada de wavelet.

  * La entropía de la imagen. Es una medida de la incertidumbre o desorden en la distribución de los valores de los píxeles en la imagen.

Los datos tienen las siguientes etiquetas:

  * La clase de la imagen, que es un valor entero que indica si la imagen es auténtica (clase 0) o falsa (clase 1).

###Dataset Dos. 

Los atributos describen las siguientes características:

* Fecha y hora en que se tomó la medición. Proporciona información temporal sobre las mediciones.

* Temperatura en grados Celsius. La temperatura es una medida de la cantidad de calor en el ambiente y afecta el nivel de comodidad de las personas.

* Humedad relativa en porcentaje (%). La humedad relativa es una medida de la cantidad de vapor de agua en el ambiente y afecta el nivel de comodidad de las personas.

* Luz en lux. La luz se mide en lux y representa la cantidad de luz visible en el ambiente.

* CO2 en partes por millón (ppm). El CO2 se mide en ppm y es una medida de la calidad del aire interior.

* Razón de humedad en kg de vapor de agua por kg de aire. La razón de humedad es una medida de la cantidad de vapor de agua en el aire y se utiliza para calcular otras medidas de humedad.

  Los datos tiene las siguientes etiquetas:

* Ocupación. Indica si el ambiente está ocupado o no. Esta variable es importante para estudiar la relación entre la ocupación y las medidas del ambiente. 

## Procesamiento de los datos

Antes de responder el resto de las preguntas, vamos a organizar y cargar los datasets en dataframes para que sea más facil trabajar los datos.

In [ ]:
import pandas as pd

### Dataset 1

In [ ]:
# Cargamos y organizamos el dataset 1
data_bank = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00267/data_banknote_authentication.txt', header=None)
data_bank.columns = ["variance", "skewness", "curtosis", "entropy", "class"]

# Remplazamos los 0 por -1 para la SVM
data_bank["class"] = data_bank["class"].replace([0],-1)

# Separamos los atributos de las etiquetas
# Atributos 
X_data_bank = data_bank[["variance", "skewness", "curtosis", "entropy"]]
#Etiquetas
y_data_bank = data_bank["class"]

In [ ]:
X_data_bank.head()

,variance,skewness,curtosis,entropy
0,3.62160,8.6661,-2.8073,-0.44699
1,4.54590,8.1674,-2.4586,-1.46210
2,3.86600,-2.6383,1.9242,0.10645
3,3.45660,9.5228,-4.0112,-3.59440
4,0.32924,-4.4552,4.5718,-0.98880


In [ ]:
y_data_bank.head()

0   -1
1   -1
2   -1
3   -1
4   -1
Name: class, dtype: int64

### Dataset 2

In [ ]:
#En este caso como el archivo es un Zip hay que hacer un par adicional de pasos
import urllib
import zipfile

# Descargamos y descomprimos los datos
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00357/occupancy_data.zip"
extract_dir = "occupancy"

zip_path, _ = urllib.request.urlretrieve(url)
with zipfile.ZipFile(zip_path, "r") as f:
    f.extractall(extract_dir)

In [ ]:
# Como los datos están separados de una vez para
# entrenar y hacer testing del modelo, es necesario 
# crear varios dataframes

def occupancy_df_process(path):
  data = pd.read_csv(path, sep=",", 
            names=["col","date","Temperature","Humidity",
                   "Light","CO2","HumidityRatio","Occupancy"])[1:]

  data = data.drop(["col"],axis=1)
  #ya no usamos date, pero pues el código está lindo 
  data['date'] = data['date'].apply(lambda x: pd.to_datetime(x).to_julian_date())
  data["Light"] = data["Light"].astype(float)
  data["Temperature"] = data["Temperature"].astype(float)
  data["Humidity"] = data["Humidity"].astype(float)
  data["CO2"] = data["CO2"].astype(float)
  data["HumidityRatio"] = data["HumidityRatio"].astype(float)
  data["Occupancy"] = data["Occupancy"].astype(float)


  # Remplazamos los 0 por -1 para la SVM
  data["Occupancy"] = data["Occupancy"].replace([0],-1)

  # Etiquetas
  X_data = data[["Temperature","Humidity",
                   "Light","CO2","HumidityRatio"]]
  # Atributos
  Y_data = data["Occupancy"]
  return X_data, Y_data

occupancy_train_X , occupancy_train_y = occupancy_df_process("occupancy/datatraining.txt")
occupancy_test_X , occupancy_test_y = occupancy_df_process("occupancy/datatest.txt")
occupancy_test2_X , occupancy_test2_y = occupancy_df_process("occupancy/datatest2.txt")

In [ ]:
occupancy_train_X.dtypes

Temperature      float64
Humidity         float64
Light            float64
CO2              float64
HumidityRatio    float64
dtype: object

In [ ]:
print(occupancy_train_y)

{1.0, -1.0}


## Punto 3)

### a) 
Para determinar si los dataset son separables, vamos a usar el algoritmo Percetron, ya que si este algoritmo converge en una cantidad finita de pasos, entonces nuestro dataset es separable. 


In [ ]:
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split

In [ ]:
def perceptron_test(X_train, y_train):
  # Crear una instancia del clasificador Perceptron
  clf = Perceptron()

  # Entrenar el modelo con el conjunto de datos de entrenamiento
  clf.fit(X_train, y_train)
  score = clf.score(X_train, y_train)
  print(score)

  # Verificar si el conjunto de datos es linealmente separable
  if clf.score(X_train, y_train) <= 9.8:
      print("El conjunto de datos es linealmente separable.")
      return True
  else:
      print("El conjunto de datos no es linealmente separable.")
      return False 


### Dataset 1

In [ ]:
# Test del percetron 
perceptron_test(X_data_bank.to_numpy(),y_data_bank.to_numpy())

0.9883381924198251
El conjunto de datos es linealmente separable.


True

### Dataset 2

In [ ]:
# Test del percetron 
perceptron_test(occupancy_train_X, occupancy_train_y )

0.9815792705391133
El conjunto de datos es linealmente separable.


True

En este caso podemos evidenciar que ninguno de los dos dataset son linealmente separables si seguimos estrictamente la definición, ya que no pudieron ser separados mediante el algoritmo Perceptron, sin embargo recordemos que los datos son muestras del mundo real, por lo que es de esperarse algún tipo de error en las mediciones. Por lo tanto al tener una cantidad tan baja de elementos que no podemos separar (menos del 2%) podemos considerar que ambos datasets son linealmete separables. 

### b) 
No es posible determinar si los datos del dataset son tomados de manera aleatoria, sin embargo confiamos y partimos de eses hecho para poder hacer el resto del análsis y crear modelos entrenados con esos datos 

### c) 
Como regla general, se recomienda que la muestra de datos tenga al menos cientos o miles de observaciones, dependiendo de la cantidad de características y el grado de complejidad del problema. Como nuestros datasets tienen miles de datos, podemos pensar que dada esta cantidad de información es posible que algún modelo pueda llegar a una generalización. 

## Punto 4) 

Para entender cómo opera el siguiente código, vamos describir brevemente dentro del código que hace cada una de sus partes mediante el uso de los comentarios de python. 

In [ ]:
import numpy as np

class SVM:
    
    def __init__(self, C = 1.0):
        # C es el parámetro de error
        self.C = C
        self.w = 0
        self.b = 0

    # Función de Pérdida Hinge / Cálculo
    def hingeloss(self, w, b, x, y):
        # Término de Regularización
        reg = 0.5 * (w * w)

        for i in range(x.shape[0]):
            # Término de Optimización
            opt_term = y[i] * ((np.dot(w, x[i])) + b)

            # Calculando la pérdida
            loss = reg + self.C * max(0, 1-opt_term)
        return loss[0][0]

    def fit(self, X, Y, batch_size=100, learning_rate=0.001, epochs=1000):
        # Número de características en X
        number_of_features = X.shape[1]

        # Número de muestras en X
        number_of_samples = X.shape[0]

        c = self.C

        # Creando ids desde 0 hasta número de muestras - 1
        ids = np.arange(number_of_samples)

        # Mezclando las muestras al azar
        np.random.shuffle(ids)

        # Creando un arreglo de ceros
        w = np.zeros((1, number_of_features))
        b = 0
        losses = []

        # Lógica del Descenso de Gradiente
        for i in range(epochs):
            # Calculando la Función de Pérdida Hinge
            l = self.hingeloss(w, b, X, Y)

            # Añadiendo todas las pérdidas
            losses.append(l)
            
            # Iniciando desde 0 hasta el número de muestras con batch_size como intervalo
            for batch_initial in range(0, number_of_samples, batch_size):
                gradw = 0
                gradb = 0

                for j in range(batch_initial, batch_initial+ batch_size):
                    if j < number_of_samples:
                        x = ids[j]
                        ti = Y[x] * (np.dot(w, X[x].T) + b)

                        if ti > 1:
                            gradw += 0
                            gradb += 0
                        else:
                            # Calculando los gradientes

                            # respectivos a w 
                            gradw += c * Y[x] * X[x]
                            # respectivos a b
                            gradb += c * Y[x]

                # Actualizando pesos y bias
                w = w - learning_rate * w + learning_rate * gradw
                b = b + learning_rate * gradb
        
        self.w = w
        self.b = b

        return self.w, self.b, losses

    def predict(self, X):
        
        prediction = np.dot(X, self.w[0]) + self.b # w.x + b
        return np.sign(prediction)

## Punto 5)

La clave del éxito de SVM radica en su capacidad para encontrar un hiperplano óptimo que tenga la mayor distancia posible entre los datos de diferentes clases. Esta distancia se conoce como margen y se mide perpendicularmente al hiperplano. El hiperplano óptimo se encuentra mediante la maximización de este margen.

### Dataset 1

In [ ]:
# Creamos la SVM 
bank_SVM = SVM()
#Partimos los datos usando sklearn 
X_train, X_test,y_train, y_test = train_test_split(X_data_bank, y_data_bank ,
                                   random_state=104, 
                                   test_size=0.25, 
                                   shuffle=True)
#Entrenamos la SVM
bank_SVM.fit(X_train.to_numpy(), y_train.to_numpy()); 

#Ahora evaluamos el rendimiento
predicted_data = bank_SVM.predict(X_test.to_numpy())

errors = len([i for i in (predicted_data - y_test) if i != 0]) 

score = (len(y_test) - errors)/len(y_test)
print(score)


0.9825072886297376


Note que este el codigó logra clasificar correctamente el 98% de datos en el primer dataset, pese a que este dataset no es linealmente separable. 

### Dataset 2

In [ ]:
#Dataset a usar
#occupancy_train_X , occupancy_train_y 
#occupancy_test_X , occupancy_test_y 
# occupancy_test2_X , occupancy_test2_y


# Creamos la SVM 
Occupancy_SVM = SVM()

# Los datos ya están partidos en secciones de prueba
# y test

#Entrenamos la SVM
Occupancy_SVM.fit(occupancy_train_X.to_numpy(), occupancy_train_y.to_numpy())

#Ahora evaluamos el rendimiento
predicted_data = Occupancy_SVM.predict(occupancy_test_X.to_numpy())

errors = len([i for i in (predicted_data - occupancy_test_y) if i != 0]) 
score = (len(occupancy_test_y) - errors)/len(occupancy_test_y)
print(score)

0.9711069418386492


## Punto 6)

Para este punto vamos realizar cross validation que es una técnica utilizada en el aprendizaje automático para evaluar el rendimiento de un modelo de predicción. La idea principal es dividir los datos en conjuntos de entrenamiento y prueba, donde el conjunto de entrenamiento se utiliza para entrenar el modelo y el conjunto de prueba se utiliza para evaluar su capacidad para generalizar a nuevos datos.

### Dataset 1


In [ ]:
perf = []
for _ in range(0,10):
  # Creamos la SVM 
  bank_SVM = SVM()
  #Partimos los datos usando sklearn 
  X_train, X_test,y_train, y_test = train_test_split(X_data_bank, y_data_bank ,
                                    random_state=104, 
                                    test_size=0.25, 
                                    shuffle=True)
  #Entrenamos la SVM
  bank_SVM.fit(X_train.to_numpy(), y_train.to_numpy()); 

  #Ahora evaluamos el rendimiento
  predicted_data = bank_SVM.predict(X_test.to_numpy())

  errors = len([i for i in (predicted_data - y_test) if i != 0]) 

  score = (len(y_test) - errors)/len(y_test)
  perf.append(score)

In [ ]:
total_score = sum(perf)/len(perf)
print(f"Resultado del cross validation {total_score}%")

Resultado del cross validation 0.985131195335277


### Dataset 2

Como los datos viene previamente en separados para el training y el testing, para sacar diferentes subconjutos vamos a pegar ambos dataframes y luego usaremos una libreria para gener los subconjuntos.

In [ ]:
#Dataset a usar
#occupancy_train_X , occupancy_train_y 
#occupancy_test_X , occupancy_test_y 

#Nuevos datos
ocuppancy_X = occupancy_train_X.append(occupancy_test_X)
occupancy_y = occupancy_train_y.append(occupancy_test_y)

In [ ]:
for _ in range(0,10):
  # Separamos los datos
  X_train, X_test,y_train, y_test = train_test_split(ocuppancy_X, occupancy_y,
                                    random_state=104, 
                                    test_size=0.25, 
                                    shuffle=True)

  # Creamos la SVM 
  Occupancy_SVM = SVM()
  #Entrenamos la SVM
  Occupancy_SVM.fit(X_train.to_numpy(), y_train.to_numpy())

  #Ahora evaluamos el rendimiento
  predicted_data = Occupancy_SVM.predict(X_test.to_numpy())

  errors = len([i for i in (predicted_data -  y_test) if i != 0]) 
  score = (len(y_test) - errors)/len(y_test)
  perf.append(score)

In [ ]:
total_score = sum(perf)/len(perf)
print(f"Resultado del cross validation {total_score}%")

Resultado del cross validation 0.8706669751959148%


Con lo anterior evidenciamos que nuestro algoritmo crea generalización, ya que independientemete de los datos que se usen para training y test, el promedio de aciertos está por encima del 98% para el primer modelo, y de 87% para el segúndo dataset. 